# 08 迁移学习

> 站在巨人的肩膀上：使用预训练模型

---

## 📚 学习目标

- [ ] 理解迁移学习的原理
- [ ] 使用 PyTorch 预训练模型
- [ ] 掌握特征提取和微调两种方法

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

try:
    import torch
    import torch.nn as nn
    import torchvision.models as models
    import torchvision.transforms as transforms
    HAS_TORCH = True
except ImportError:
    HAS_TORCH = False
    print("请安装 PyTorch 和 torchvision")

---

## 💡 为什么迁移学习有效？

### CNN 的特征层次

```
浅层（靠近输入）      深层（靠近输出）
     ↓                    ↓
  边缘、颜色    →    纹理、形状    →    高级语义
     ↓                    ↓                ↓
  通用特征            领域特定         任务特定
```

**关键洞察**：浅层特征（边缘、纹理）在不同任务中是通用的！

### 迁移学习策略

1. **特征提取 (Feature Extraction)**：冻结预训练层，只训练新的分类头
2. **微调 (Fine-tuning)**：解冻部分/全部层，用小学习率继续训练

---

## 💻 加载预训练模型

In [ ]:
if HAS_TORCH:
    # 加载预训练的 ResNet-18
    # weights='DEFAULT' 使用 ImageNet 预训练权重
    resnet = models.resnet18(weights='DEFAULT')
    
    print("ResNet-18 结构（最后几层）:")
    print("...")
    for name, module in list(resnet.named_children())[-3:]:
        print(f"{name}: {module}")
    
    print(f"\n原始分类头: {resnet.fc.in_features} → {resnet.fc.out_features} (ImageNet 1000类)")

---

## 💻 方法1：特征提取

In [ ]:
if HAS_TORCH:
    # 复制模型
    model_feature_extract = models.resnet18(weights='DEFAULT')
    
    # 冻结所有层
    for param in model_feature_extract.parameters():
        param.requires_grad = False
    
    # 替换分类头（假设新任务是 5 分类）
    num_classes = 5
    num_features = model_feature_extract.fc.in_features
    model_feature_extract.fc = nn.Linear(num_features, num_classes)
    
    # 统计可训练参数
    trainable_params = sum(p.numel() for p in model_feature_extract.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model_feature_extract.parameters())
    
    print("特征提取模式:")
    print(f"  总参数: {total_params:,}")
    print(f"  可训练参数: {trainable_params:,}")
    print(f"  冻结比例: {(1 - trainable_params/total_params)*100:.1f}%")
    print(f"\n💡 只需要训练最后一层！")

---

## 💻 方法2：微调

In [ ]:
if HAS_TORCH:
    # 复制模型
    model_finetune = models.resnet18(weights='DEFAULT')
    
    # 冻结前面的层（layer1, layer2）
    for name, param in model_finetune.named_parameters():
        if 'layer3' not in name and 'layer4' not in name and 'fc' not in name:
            param.requires_grad = False
    
    # 替换分类头
    model_finetune.fc = nn.Linear(model_finetune.fc.in_features, num_classes)
    
    # 统计可训练参数
    trainable_params = sum(p.numel() for p in model_finetune.parameters() if p.requires_grad)
    
    print("微调模式 (解冻 layer3, layer4, fc):")
    print(f"  可训练参数: {trainable_params:,}")
    
    # 列出可训练的层
    print("\n可训练的层:")
    for name, param in model_finetune.named_parameters():
        if param.requires_grad:
            print(f"  {name}")

---

## 💻 完整的迁移学习流程

In [ ]:
if HAS_TORCH:
    # 数据预处理（ImageNet 标准化）
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
    
    print("数据预处理流程:")
    print("1. Resize to 256")
    print("2. CenterCrop to 224×224")
    print("3. ToTensor (0-255 → 0-1)")
    print("4. Normalize (ImageNet 均值和标准差)")

In [ ]:
if HAS_TORCH:
    # 训练配置
    def get_optimizer(model, mode='feature_extract'):
        """
        根据迁移学习模式配置优化器
        
        - feature_extract: 只训练 fc 层
        - finetune: 不同层使用不同学习率
        """
        if mode == 'feature_extract':
            # 只优化 fc 层
            return torch.optim.Adam(model.fc.parameters(), lr=1e-3)
        
        elif mode == 'finetune':
            # 不同层使用不同学习率
            # 越靠近输出的层，学习率越高
            param_groups = [
                {'params': model.layer3.parameters(), 'lr': 1e-5},
                {'params': model.layer4.parameters(), 'lr': 1e-4},
                {'params': model.fc.parameters(), 'lr': 1e-3},
            ]
            return torch.optim.Adam(param_groups)
    
    print("优化器配置:")
    print("  特征提取: fc 层 lr=1e-3")
    print("  微调: layer3 lr=1e-5, layer4 lr=1e-4, fc lr=1e-3")
    print("\n💡 越靠近输入的层，学习率越小（保留通用特征）")

---

## 📝 本章小结

### 迁移学习策略对比

| 策略 | 可训练参数 | 训练速度 | 适用场景 |
|------|-----------|---------|----------|
| 特征提取 | 极少 | 快 | 小数据集，任务相似 |
| 微调 | 较多 | 较慢 | 中等数据集，任务有差异 |
| 从零训练 | 全部 | 慢 | 大数据集，任务很不同 |

### 实用建议

1. **数据少**：用特征提取
2. **数据多**：用微调
3. **域差异大**：解冻更多层
4. **域差异小**：冻结更多层

---

## 🚀 下一步

继续学习 **09_feature_visualization.ipynb** - 特征可视化